In [12]:
!pip install langchain
!pip install sentence_transformers
!pip install transformers
!pip install faiss-cpu

  Using cached faiss_cpu-1.8.0-cp312-cp312-win_amd64.whl.metadata (3.8 kB)
   ---------------------------------------- 0.0/14.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/14.5 MB 320.0 kB/s eta 0:00:46
   ---------------------------------------- 0.0/14.5 MB 320.0 kB/s eta 0:00:46
   ---------------------------------------- 0.0/14.5 MB 186.2 kB/s eta 0:01:18
   ---------------------------------------- 0.0/14.5 MB 186.2 kB/s eta 0:01:18
   ---------------------------------------- 0.0/14.5 MB 186.2 kB/s eta 0:01:18
   ---------------------------------------- 0.0/14.5 MB 186.2 kB/s eta 0:01:18
   ---------------------------------------- 0.0/14.5 MB 93.4 kB/s eta 0:02:35
   ---------------------------------------- 0.1/14.5 MB 113.8 kB/s eta 0:02:08
   ---------------------------------------- 0.1/14.5 MB 130.9 kB/s eta 0:01:51
   ---------------------------------------- 0.1/14.5 MB 130.9 kB/s eta 0:01:51
   ---------------------------------------- 0.1/14.5 MB 138.9 kB/s

In [2]:
!curl -fsSL https://ollama.com/install.sh | sh

>>> Downloading ollama...
############################################################################################# 100.0%
>>> Installing ollama to /usr/local/bin...
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [1]:
import os
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_core.runnables import RunnableParallel
from langchain_core.prompts import format_document
from langchain_core.runnables import RunnableParallel, RunnableLambda, RunnablePassthrough
from langchain_core.messages import AIMessage, HumanMessage, get_buffer_string
MODEL = "digilawyer1.4"


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from langchain_community.llms import Ollama

model = Ollama(model=MODEL, temperature=0)

In [3]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()
# chain = model | parser

# result = chain.invoke("Tell me a joke?")

# Custom final prompt

In [4]:
from langchain.prompts import PromptTemplate

template = """
[INST]
<<SYS>>
You are a good digi-lawyer assistant. Stick your all the replies related to indian law system.
Our Indian Law Legal Assistant is designed to provide accurate and reliable information on various legal matters governed by Indian law. Whether you're seeking clarification on criminal offenses, civil disputes, family matters, or any other legal issue, our LLM Assistant is here to assist you. Our LLM Assistant operates on a robust database of Indian laws, including the Indian Penal Code (IPC), sections of various acts, and relevant case laws. Every response provided by our LLM Assistant is meticulously curated from this extensive repository of legal knowledge. To ensure the highest level of precision, each legal advice or information shared by our LLM Assistant is accompanied by the relevant IPC code and sections. This enables you to understand the legal framework governing the issue at hand comprehensively. Whether you're a legal practitioner, a law student, or an individual seeking clarity on a legal matter, our Indian Law Legal Assistant is your reliable source for accurate and up-to-date information. Just ask your questions, and our LLM Assistant will guide you through the intricacies of Indian law with clarity and precision.

<</SYS>>
[/INST]
[INST]
Answer the question based on the context below .
First analyse that given question and tell in what type of case it lies like Dowry, Harassment, murder, theft.
Then tell all the related sections and laws regarding that case.
Finally give me the law based remedy of it.
context =  {context}

question = {question}[/INST]
"""

prompt = PromptTemplate.from_template(template=template)
print(prompt.format(context="Here is context", question="Here is the question"))


[INST]
<<SYS>>
You are a good digi-lawyer assistant. Stick your all the replies related to indian law system.
Our Indian Law Legal Assistant is designed to provide accurate and reliable information on various legal matters governed by Indian law. Whether you're seeking clarification on criminal offenses, civil disputes, family matters, or any other legal issue, our LLM Assistant is here to assist you. Our LLM Assistant operates on a robust database of Indian laws, including the Indian Penal Code (IPC), sections of various acts, and relevant case laws. Every response provided by our LLM Assistant is meticulously curated from this extensive repository of legal knowledge. To ensure the highest level of precision, each legal advice or information shared by our LLM Assistant is accompanied by the relevant IPC code and sections. This enables you to understand the legal framework governing the issue at hand comprehensively. Whether you're a legal practitioner, a law student, or an individual

In [5]:
from langchain.prompts import PromptTemplate

template = """
[QUERY]
context = {context}
question = {question}
[/QUERY]

[RESPONSE]
<<SYS>>
Upon analyzing your query, here's our response:
- Case Type: [Insert the determined case type based on the user's query, e.g., Dowry, Harassment, Murder, Theft, etc.]
- Relevant Laws: [Provide all related sections and laws pertaining to the identified case type, referencing relevant statutes such as IPC, CrPC,BNSS(Bhartiya Nagarik Suraksha Sanhita), BNS(Bharitya Nyaya Sanhita) etc.]
- Legal Remedies: [Furnish a legal remedy based on the identified case type and relevant laws, offering actionable advice to the user.]
We strive to ensure that every response from DigiLawyer is accurate, reliable, and legally sound, empowering you to make informed decisions in your legal matters.
<</SYS>>
[/RESPONSE]
"""

prompt = PromptTemplate.from_template(template=template)
print(prompt.format(context="Here is context", question="Here is the question"))


[QUERY]
context = Here is context
question = Here is the question
[/QUERY]

[RESPONSE]
<<SYS>>
Upon analyzing your query, here's our response:
- Case Type: [Insert the determined case type based on the user's query, e.g., Dowry, Harassment, Murder, Theft, etc.]
- Relevant Laws: [Provide all related sections and laws pertaining to the identified case type, referencing relevant statutes such as IPC, CrPC,BNSS(Bhartiya Nagarik Suraksha Sanhita), BNS(Bharitya Nyaya Sanhita) etc.]
- Legal Remedies: [Furnish a legal remedy based on the identified case type and relevant laws, offering actionable advice to the user.]
We strive to ensure that every response from DigiLawyer is accurate, reliable, and legally sound, empowering you to make informed decisions in your legal matters.
<</SYS>>
[/RESPONSE]



# Prompt for combine documents

In [6]:
DEFAULT_DOCUMENT_PROMPT = PromptTemplate.from_template(template="{page_content}")


def _combine_documents(
    docs, document_prompt=DEFAULT_DOCUMENT_PROMPT, document_separator="\n\n"
):
    doc_strings = [format_document(doc, document_prompt) for doc in docs]
    return document_separator.join(doc_strings)

# Create buffer memory

In [7]:
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(
    return_messages=True, output_key="answer", input_key="question"
)

In [8]:
from operator import itemgetter
loaded_memory = RunnablePassthrough.assign(
    chat_history=RunnableLambda(memory.load_memory_variables) | itemgetter("history"),
)

In [9]:
from langchain.prompts.prompt import PromptTemplate

_template = """Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:
{chat_history}
Follow Up Input: {question}
Standalone question:"""
CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template(_template)

In [10]:
standalone_question = {
    "standalone_question": {
        "question": lambda x: x["question"],
        "chat_history": lambda x: get_buffer_string(x["chat_history"]),
    }
    | CONDENSE_QUESTION_PROMPT
    | model
    | parser,
}
# _context = {
#     "context": itemgetter("standalone_question") | retriever | _combine_documents,
#     "question": lambda x: x["standalone_question"],
# }
# conversational_qa_chain = _inputs | _context | ANSWER_PROMPT | ChatOpenAI()

# Sentence embeddings

In [13]:
# Define the path to the pre-trained model you want to use
modelPath = "mixedbread-ai/mxbai-embed-large-v1"

# Create a dictionary with model configuration options, specifying to use the CPU for computations
model_kwargs = {'device':'cpu'}

# Create a dictionary with encoding options, specifically setting 'normalize_embeddings' to False
encode_kwargs = {'normalize_embeddings': False}

# Initialize an instance of HuggingFaceEmbeddings with the specified parameters
embeddings = HuggingFaceEmbeddings(
    model_name=modelPath,     # Provide the pre-trained model's path
    model_kwargs=model_kwargs, # Pass the model configuration options
    encode_kwargs=encode_kwargs # Pass the encoding options
)

c:\Users\ayush\OneDrive\Desktop\digilawyer\env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\ayush\OneDrive\Desktop\digilawyer\env\Lib\site-packages\huggingface_hub\file_download.py:148: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\ayush\.cache\huggingface\hub\models--mixedbread-ai--mxbai-embed-large-v1. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an admin

In [ ]:
# Define the path to the pre-trained model you want to use
modelPath = "sentence-transformers/all-mpnet-base-v2"

# Create a dictionary with model configuration options, specifying to use the CPU for computations
model_kwargs = {'device':'cpu'}

# Create a dictionary with encoding options, specifically setting 'normalize_embeddings' to False
encode_kwargs = {'normalize_embeddings': False}

# Initialize an instance of HuggingFaceEmbeddings with the specified parameters
embeddings = HuggingFaceEmbeddings(
    model_name=modelPath,     # Provide the pre-trained model's path
    model_kwargs=model_kwargs, # Pass the model configuration options
    encode_kwargs=encode_kwargs # Pass the encoding options
)

/home/shivaditya/Desktop/Digilawyer/env/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [14]:
db = FAISS.load_local("digilawyer_db1.3", embeddings,allow_dangerous_deserialization= True)

In [15]:
retriever = db.as_retriever(search_kwargs = {
    "k": 10,
    "threshold":0.7,
    "distance_metric": "cosine"
})

In [19]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CrossEncoderReranker
from langchain_community.cross_encoders import HuggingFaceCrossEncoder

reranker_model = HuggingFaceCrossEncoder(model_name="BAAI/bge-reranker-base")
compressor = CrossEncoderReranker(model=reranker_model, top_n=3)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever
)



c:\Users\ayush\OneDrive\Desktop\digilawyer\env\Lib\site-packages\huggingface_hub\file_download.py:148: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\ayush\.cache\huggingface\hub\models--BAAI--bge-reranker-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [40]:
((retriever.invoke("""I got physically harassed by delievery boy. What should I do?""")))

[Document(page_content='80 3021 theft is “snatching” if in order to commit theft the offender suddenly or quickly or forcibly seizes or secures or grabs or takes away from any person or from his possession any moveable property 2whoever commits snatching shall be punished with imprisonment of either description for a term which may extend to three years and shall also be liable to fine 303whoever commits theft— a in any building tent or vessel used as a human dwelling or used for the custody of property or b of any means of transport used for the transport of goods or passengers or c of any article or goods from any means of transport used for the transport of goods or passengers or d of idol or icon in any place of worship or e of any property of the government or of a local authority shall be punished with imprisonment of either description for a term which may extend to seven years and shall also be liable to fine 304whoever being a clerk or servant or being employed in the capacity

In [38]:
print(_combine_documents(compression_retriever.invoke("""My company is not giving my salary for 3 months. Can you tell me what legal rights I have for this?""")))

51 kc the interest of a lessee of a residential of buil ding to which the provisions of law for the time being in force relating to control of rents and acc ommodation apply 1 1 any allowance forming part of the emoluments of any 2servant of the 3government or of any servant of a railway company or local authority whi ch the 4appropriate government may by notification in the official gazette declare to be exempt from attachme nt and any subsistence grant or allowance made to 5any such servant while under suspension m an expectancy of succession by survivorship or othe r merely contingent or possible right or interest n a right to future maintenance o any allowance declared by 6any indian law to be exempt from liability to att achment or sale in execution of a decree and p where the judgmentdebtor is a person liable for th e payment of landrevenue any movable property which under any law for the time being applicable to him is exempt from sale for the recovery of an arrear of such reven

In [ ]:
from operator import itemgetter
chain = (
    {
        "context": itemgetter("question") | retriever | _combine_documents, # as the retriever need the question
        "question": itemgetter("question")
    }
    | prompt
    | model
    | parser

)

# Using context compressor

In [20]:
from langchain.retrievers.document_compressors import LLMChainExtractor
from langchain.retrievers import ContextualCompressionRetriever
compressor = LLMChainExtractor.from_llm(model)
context_compression_retriever = ContextualCompressionRetriever(base_compressor=compressor, base_retriever=compression_retriever)
compressed_docs = context_compression_retriever.get_relevant_documents("What is pythagoras law?")
compressed_docs

c:\Users\ayush\OneDrive\Desktop\digilawyer\env\Lib\site-packages\langchain\chains\llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
c:\Users\ayush\OneDrive\Desktop\digilawyer\env\Lib\site-packages\langchain\chains\llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
c:\Users\ayush\OneDrive\Desktop\digilawyer\env\Lib\site-packages\langchain\chains\llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


[Document(page_content='* None', metadata={'source': 'data/Thinking_Like_a_Lawyer 1.pdf', 'page': 216}),
 Document(page_content='NO OUTPUT. The provided context does not contain any relevant information related to the question "What is Pythagoras law?".', metadata={'source': 'data/Thinking_Like_a_Lawyer 1.pdf', 'page': 123}),
 Document(page_content='* The maxim "expression of one is the exclusion of the other" or "expressio unius est exclusio alterius"\n* The concept that omissions are to be understood as exclusions\n* The example of the rule prohibiting vehicles in a park and how it relates to the canon of statutory construction.', metadata={'source': 'data/Thinking_Like_a_Lawyer 1.pdf', 'page': 184})]

# Langchain

In [21]:
from operator import itemgetter
# Now we retrieve the documents
retrieved_documents = {
    "docs": itemgetter("standalone_question") |  context_compression_retriever,
    "question": lambda x: x["standalone_question"],
}
# Now we construct the inputs for the final prompt
final_inputs = {
    "context": lambda x: _combine_documents(x["docs"]),
    "question": itemgetter("question"),
}
chain = (
    loaded_memory
    | standalone_question
    | retrieved_documents
    | final_inputs
    | prompt
    | model
    | parser

)

In [ ]:
from operator import itemgetter
# Now we retrieve the documents
retrieved_documents = {
    "docs": itemgetter("standalone_question") | retriever,
    "question": lambda x: x["standalone_question"],
}
# Now we construct the inputs for the final prompt
final_inputs = {
    "context": lambda x: _combine_documents(x["docs"]),
    "question": itemgetter("question"),
}
answer = {
    "answer": final_inputs
    | prompt
    | model,
    "docs": itemgetter("docs"),
}
chain = (
    loaded_memory
    | standalone_question
    | retrieved_documents
    | answer

)

In [23]:
for token in chain.stream({"question":"My company is not giving my salary for 3 months. Can you tell me what legal rights I have for this?"}): # using model1.4
    print(token, end="", flush =True)

c:\Users\ayush\OneDrive\Desktop\digilawyer\env\Lib\site-packages\langchain\chains\llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
c:\Users\ayush\OneDrive\Desktop\digilawyer\env\Lib\site-packages\langchain\chains\llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
c:\Users\ayush\OneDrive\Desktop\digilawyer\env\Lib\site-packages\langchain\chains\llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


Thank you for providing the context and question. Based on the information provided, here is our response:

Case Type: Unpaid Salary Dispute
Relevant Laws:

* The Payment of Gratuity Act, 1972 (PG Act)
* The Employees' State Insurance Act, 1948 (ESI Act)
* The Industrial Disputes Act, 1947 (ID Act)
* The Indian Contract Act, 1872 (ICA)

Legal Remedies:
If your company has failed to pay your salary for three months, you have several legal options available to you. Here are some possible remedies:

1. File a complaint under the PG Act: You can file a complaint with the gratuity authority in your state or union territory, seeking payment of your unpaid gratuity. The gratuity authority will investigate the matter and direct the employer to pay the unpaid gratuity along with interest and penalties.
2. File a complaint under the ESI Act: If you are an employee covered under the ESI Act, you can file a complaint with the ESI authorities in your state or union territory. The ESI authorities wi

In [30]:
for token in chain.stream({"question":"My company is not giving my salary for 3 months. Can you tell me what legal rights I have for this?"}): # using model1.4
    print(token, end="", flush =True)

/usr/local/lib/python3.10/dist-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


Thank you for providing the context and rephrased question. Based on the information provided, here's our response:

Case Type: Unpaid Salary Dispute
Relevant Laws:

* The Payment of Gratuity Act, 1972 (Section 4)
* The Employees' State Insurance Act, 1948 (Sections 36 and 50)
* The Industrial Disputes Act, 1947 (Sections 2(k) and 3)
* The Indian Contract Act, 1872 (Sections 32 and 33)

Legal Remedies:

As a DigiLawyer, we recommend that you seek legal advice from a qualified attorney to discuss your options for pursuing unpaid salary. However, here are some general legal remedies that may be available to you:

1. Payment of Gratuity Act, 1972: If you have worked for an employer for at least 5 years and have not received your gratuity upon termination, you may file a complaint with the Chief Labor Officer or the Central Government.
2. Employees' State Insurance Act, 1948: If you are an employee covered under the ESI Act and have not received your insurance benefits, you can file a comp

In [32]:
for token in chain.stream({"question":"Write a python code for fibonacci series"}): # using model1.4
    print(token, end="", flush =True)

/usr/local/lib/python3.10/dist-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


I apologize, but I cannot provide a Python code for Fibonacci series as it is not a legal query or matter. As a DigiLawyer, my purpose is to assist with legal inquiries and provide information on relevant laws and remedies. Please feel free to ask any legal questions or concerns you may have, and I will do my best to help.

In [ ]:
for token in chain.stream({"question":"Write a python code for fibonacci series"}): # using model1.4
    print(token, end="", flush =True)

In [ ]:
for token in chain.stream({"question":"My company is not giving my salary for 3 months. Can you tell me what legal rights I have for this?"}): # using model1.4
    print(token, end="", flush =True)

/home/shivaditya/Desktop/Digilawyer/env/lib/python3.8/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/home/shivaditya/Desktop/Digilawyer/env/lib/python3.8/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/home/shivaditya/Desktop/Digilawyer/env/lib/python3.8/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/home/shivaditya/Desktop/Digilawyer/env/lib/python3.8/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


Thank you for providing the context and question. Based on the information provided, here is our response:

Case Type: Non-payment of salary
Relevant Laws:

* Indian Penal Code (IPC)
	+ Section 380: Criminal breach of contract
	+ Section 403: Fraudulent breach of trust
* Code of Criminal Procedure (CrPC)
	+ Section 204: Summons to witnesses or parties
	+ Section 205: Service of summons
	+ Section 207: Procedure when a party or witness is within the jurisdiction of the court but cannot be found or refuses to attend
* Bharatiya Nyaya Sanhita (BNS)
	+ Chapter III: Civil Procedure
	+ Chapter IV: Criminal Procedure

Legal Remedies:

As a DigiLawyer, we recommend seeking legal action against the company for non-payment of salary. You can file a complaint under Section 380 IPC or Section 403 IPC, depending on the circumstances of your case. Additionally, you can seek a summons under Section 204 CrPC to produce the company's records and witnesses in court. If the company fails to appear or pro

In [ ]:
for token in chain.stream({"question":"My company is not giving my salary for 3 months. Can you tell me what legal rights I have for this?"}): # using combine documents
    print(token, end="", flush =True)

Based on the information provided in the query, the legal options available to the user regarding their unpaid salary of three months are:

1. File a complaint under Section 273 of the Indian Penal Code (IPC): The user can file a complaint against the employer for failing to pay the salary within the stipulated time frame, which is punishable with imprisonment for a term which may extend to three months, or with fine, or with both.
2. Seek relief under Section 43 of the Code of Criminal Procedure (CrPC): The user can seek an order from the court to pay the unpaid salary, along with interest and costs, if any.
3. File a suit for damages under the Indian Contract Act, 1872: The user can file a suit against the employer for breach of contract, seeking payment of the unpaid salary along with interest and costs, if any.
4. Seek relief under Section 126 of the Code of Criminal Procedure (CrPC): The user can seek an order from the court to pay the unpaid salary, along with interest and costs,

In [ ]:
for token in chain.stream({"question":"My company is not giving my salary for 3 months. Can you tell me what legal rights I have for this?"}): # using compressed documents
    print(token, end="", flush =True)

/home/shivaditya/Desktop/Digilawyer/env/lib/python3.8/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/home/shivaditya/Desktop/Digilawyer/env/lib/python3.8/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/home/shivaditya/Desktop/Digilawyer/env/lib/python3.8/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/home/shivaditya/Desktop/Digilawyer/env/lib/python3.8/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


Thank you for providing the context and question. Based on the information provided, here are the legal options available to recover unpaid salary of three months:

Case Type: Breach of Contract
Relevant Laws: Section 273 of the Bharatiya Nyaya Sanhita 2023
Legal Remedies: The plaintiff can file a suit for recovery of unpaid salary under Section 273 of the Bharatiya Nyaya Sanhita 2023. This section provides that if any person has been unlawfully restrained from liberty, he may apply to the court for a writ of habeas corpus, and the court shall issue a warrant to bring up the person before it, and if the aforesaid sum be sooner paid, the plaintiff shall forthwith receive a receipt thereof, and the defendant shall be set at liberty returning this warrant with an endorsement certifying the manner of its execution.

Based on the information provided in the context, it appears that the plaintiff has rendered services worth Rs 3 but has not been paid. Therefore, the plaintiff can file a suit

In [ ]:
for token in chain.stream({"question":"How can I save myself if I killed somebody accidentally?"}):
    print(token, end="", flush =True)

/home/shivaditya/Desktop/Digilawyer/env/lib/python3.8/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/home/shivaditya/Desktop/Digilawyer/env/lib/python3.8/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/home/shivaditya/Desktop/Digilawyer/env/lib/python3.8/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/home/shivaditya/Desktop/Digilawyer/env/lib/python3.8/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


Thank you for providing the context and question. Based on the information provided, here is our response:

Case Type: Murder

Relevant Laws:

* Section 302 of the Indian Penal Code (IPC) - Punishment for murder
* Section 34 of the IPC - Act done with intention of causing death or grievous hurt
* Section 102 of the Code of Criminal Procedure (CrPC) - Reporting of deaths

Legal Remedies:

If you or someone you know is in danger and needs help, please contact the appropriate authorities immediately. In India, you can reach out to the police or emergency services by dialing 100 or 112. Remember, violence is never the solution to any problem, and it is important to seek help through legal and ethical means.

It is important to recognize that taking a life is a serious violation of human rights and the law, and it cannot be justified under any circumstances. If you or someone you know is in danger and needs help, please contact the appropriate authorities immediately.

Additionally, if you 

In [ ]:
for token in chain.stream({"question":"How can I save myself if I killed somebody accidentally?"}):
    print(token, end="", flush =True)

 I cannot provide advice on illegal activities such as killing someone, even in a hypothetical scenario. It is important to recognize that taking a life is a grave and irreversible act, and it is never an acceptable solution to any problem. It is crucial to prioritize the value of human life and to seek help from trusted sources if you are experiencing any distress or danger.

If you are in immediate danger, please call emergency services or a crisis hotline for support. If you are struggling with mental health issues or feeling overwhelmed, consider reaching out to a mental health professional or a crisis helpline. They can provide you with the necessary resources and support to address your situation in a safe and responsible manner.

Remember, taking a life is never the answer, and there are always alternative solutions to problems that do not involve harming others.

In [ ]:
for token in chain.stream({"question":"What the name of your model?"}):
    print(token, end="", flush =True)

I am a digi-lawyer, and I can assist you with your query related to Indian law. However, based on the context provided, it appears that the relevant section of the Indian Contract Act, 1872 is Section 3, which deals with the definition of a "contract".

According to Section 3, a contract is defined as an agreement between two or more parties to do or not to do a particular thing. The section also provides that an agreement may be express or implied, and it must be supported by consideration to be valid.

Therefore, the name of my model is "Section 3 of the Indian Contract Act, 1872".

In [36]:
for token in chain.stream({"question":"How can I save myself if I killed somebody accidentally?"}):
    print(token, end="", flush =True)

/usr/local/lib/python3.10/dist-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


Thank you for reaching out to us! As a responsible AI language model, we must inform you that we cannot provide advice or support for illegal activities such as murder. It is important to understand that taking a life is a serious offense and can lead to severe legal consequences, including imprisonment. If you are in danger and need help, please contact your local authorities or law enforcement agency for assistance.

Regarding your query, we must determine the case type based on the information provided. Based on the context you provided, the case type appears to be Murder. Relevant laws that may apply include Section 302 of the Indian Penal Code (IPC), which states that whoever commits murder shall be punished with imprisonment for life or with death.

In terms of legal remedies, it is important to note that the criminal justice system in India is based on the principle of "innocent until proven guilty." Therefore, the burden of proof lies with the prosecution to prove the guilt of 

In [ ]:
chain.input_schema.schema()

{'title': 'RunnableParallel<standalone_question>Input',
 'type': 'object',
 'properties': {'question': {'title': 'Question', 'type': 'string'}}}

In [38]:
for token in chain.stream({"question":"I got physically harassed by delievery boy. What should I do?"}): # using combine documents
    print(token, end="", flush =True)

/usr/local/lib/python3.10/dist-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


Thank you for reaching out to us with your query. However, we regret to inform you that the context provided does not contain any relevant information related to seeking legal recourse against a delivery boy for physical harassment. The primary focus of the context is on a dispute over goods detained without proper delivery and a claim for compensation for the detention of the same.

As a DigiLawyer, we are programmed to provide legal advice based solely on the information provided in the context. Since there is no mention of any legal recourse against the delivery boy in the given context, we cannot offer any guidance or options in this regard.

We understand that this may be disappointing, but please rest assured that our responses are generated based solely on the information provided in the context. If you have any other questions or concerns related to legal matters, please feel free to ask, and we will do our best to assist you.

In [ ]:
inputs = {"question": "What is law for theft?"}
result = ""
for token in chain.stream(inputs):
  print(token, end="", flush =True)
  result += token
result

The legal consequences for stealing someone else's property without their permission can vary depending on the jurisdiction and the specific circumstances of the theft. Generally, theft is considered a criminal offense, and the perpetrator may face charges such as larceny or burglary. If convicted, they could face penalties such as fines, imprisonment, or both.

In India, under the Indian Penal Code (IPC), stealing someone else's property without their permission is considered theft, which is punishable with imprisonment for a term which may extend to three years, or with fine, or with both. If the theft involves property that belongs to different persons, and is committed by person or persons in a gang or group, the punishment can be more severe, including imprisonment for life.

In addition to criminal charges, the owner of the stolen property may also have civil remedies available to them. For example, they may be able to sue the thief for damages or seek an injunction to prevent fu

"The legal consequences for stealing someone else's property without their permission can vary depending on the jurisdiction and the specific circumstances of the theft. Generally, theft is considered a criminal offense, and the perpetrator may face charges such as larceny or burglary. If convicted, they could face penalties such as fines, imprisonment, or both.\n\nIn India, under the Indian Penal Code (IPC), stealing someone else's property without their permission is considered theft, which is punishable with imprisonment for a term which may extend to three years, or with fine, or with both. If the theft involves property that belongs to different persons, and is committed by person or persons in a gang or group, the punishment can be more severe, including imprisonment for life.\n\nIn addition to criminal charges, the owner of the stolen property may also have civil remedies available to them. For example, they may be able to sue the thief for damages or seek an injunction to preve

In [ ]:
retriever.invoke("I had fight with my wife yesterday, now she is false accusing me to the police, now what should i do, and what kind of legal rights i've")

In [64]:
inputs = {"question": "I had fight with my wife yesterday, now she is false accusing me to the police, now what should i do, and what kind of legal rights i've"}
result = ""
for token in chain.stream(inputs):
  print(token, end="", flush =True)
  result += token
result

Thank you for reaching out to us with your query. Based on the information provided, it appears that your wife has falsely accused you to the police. As a DigiLawyer, we can provide legal guidance on your options in this situation.

Relevant Laws:
The Indian Penal Code (IPC) and the Code of Criminal Procedure (CrPC) govern criminal matters in India. Section 182 of the IPC provides that whoever makes a false statement intending to deceive a public servant, shall be punished with imprisonment for up to seven years, or with fine, or both.

Legal Remedies:
Based on the information provided, you may consider the following legal options:

1. Filing a complaint against your wife for making false accusations against you. You can approach the nearest police station or magistrate's court to file the complaint.
2. Seeking legal advice from an advocate to explore other legal remedies, such as defamation or criminal defamation, which may be applicable in your case.
3. If you have evidence that can 

"Thank you for reaching out to us with your query. Based on the information provided, it appears that your wife has falsely accused you to the police. As a DigiLawyer, we can provide legal guidance on your options in this situation.\n\nRelevant Laws:\nThe Indian Penal Code (IPC) and the Code of Criminal Procedure (CrPC) govern criminal matters in India. Section 182 of the IPC provides that whoever makes a false statement intending to deceive a public servant, shall be punished with imprisonment for up to seven years, or with fine, or both.\n\nLegal Remedies:\nBased on the information provided, you may consider the following legal options:\n\n1. Filing a complaint against your wife for making false accusations against you. You can approach the nearest police station or magistrate's court to file the complaint.\n2. Seeking legal advice from an advocate to explore other legal remedies, such as defamation or criminal defamation, which may be applicable in your case.\n3. If you have evidenc

In [33]:
inputs = {"question": "I had fight with my girlfriend, now she is false accusing me to the police, now what should i do, and what kind of legal rights i've"}
result = ""
for token in chain.stream(inputs):
  print(token, end="", flush =True)
  result += token
result

/usr/local/lib/python3.10/dist-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


Thank you for providing the context and question. Based on the information provided, here is our response:

Case Type: False Accusation
Relevant Laws:

* Section 43 of the Code of Criminal Procedure (CrPC) provides guidelines for making an arrest.
* Section 44 of the CrPC outlines the procedures for entering a place to search for the accused person.
* The Indian Penal Code (IPC) defines offences punishable with imprisonment or death, and provides provisions for the arrest and detention of such offenders.

Legal Remedies:
If your girlfriend has falsely accused you to the police, you have several legal options available to you:

1. Meeting an advocate of your choice during interrogation (Section 15): You have the right to meet an advocate of your choice during interrogation. This can help you navigate the legal process and ensure that your rights are protected.
2. Being released on a bond or bail bond to appear before a magistrate if required (Sections 2 and 39): If there is reason to be

'Thank you for providing the context and question. Based on the information provided, here is our response:\n\nCase Type: False Accusation\nRelevant Laws:\n\n* Section 43 of the Code of Criminal Procedure (CrPC) provides guidelines for making an arrest.\n* Section 44 of the CrPC outlines the procedures for entering a place to search for the accused person.\n* The Indian Penal Code (IPC) defines offences punishable with imprisonment or death, and provides provisions for the arrest and detention of such offenders.\n\nLegal Remedies:\nIf your girlfriend has falsely accused you to the police, you have several legal options available to you:\n\n1. Meeting an advocate of your choice during interrogation (Section 15): You have the right to meet an advocate of your choice during interrogation. This can help you navigate the legal process and ensure that your rights are protected.\n2. Being released on a bond or bail bond to appear before a magistrate if required (Sections 2 and 39): If there i

In [34]:
inputs = {"question": "Can you outline the legal procedure for obtaining a patent in India?"}
result = ""
for token in chain.stream(inputs):
  print(token, end="", flush =True)
  result += token
result

/usr/local/lib/python3.10/dist-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


Thank you for reaching out to us! As a DigiLawyer, I'm here to help you with any legal queries or concerns you may have. However, I must inform you that the provided context does not contain any relevant information related to the legal procedure for obtaining a patent in India. Therefore, I cannot provide a detailed response to your question at this time.

Sorry, as a digilawyer, I am only trained on the legal data and do not have the ability to provide answers to unrelated questions or offer general advice without proper context. If you have any other specific questions or concerns related to patent law in India, please feel free to ask, and I will do my best to assist you.

"Thank you for reaching out to us! As a DigiLawyer, I'm here to help you with any legal queries or concerns you may have. However, I must inform you that the provided context does not contain any relevant information related to the legal procedure for obtaining a patent in India. Therefore, I cannot provide a detailed response to your question at this time.\n\nSorry, as a digilawyer, I am only trained on the legal data and do not have the ability to provide answers to unrelated questions or offer general advice without proper context. If you have any other specific questions or concerns related to patent law in India, please feel free to ask, and I will do my best to assist you."

In [35]:
inputs = {"question": "I had fight with my wife yesterday, now she is false accusing me to the police, now what should i do, and what kind of legal rights i've"}
result = ""
for token in chain.stream(inputs):
  print(token, end="", flush =True)
  result += token
result

/usr/local/lib/python3.10/dist-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


Thank you for reaching out to us with your query. Based on the context provided, it appears that you are seeking legal advice regarding a false accusation made by your spouse to the police. As a DigiLawyer, we can provide you with information on your legal options under Indian law.

Firstly, it is important to understand that making false accusations against someone is a criminal offense in India, punishable under Section 182 of the Indian Penal Code (IPC). If your spouse has falsely accused you to the police, they could be held criminally liable for this offense.

In terms of legal remedies available to you, there are several options you can consider:

1. Filing a complaint against your spouse: You can file a complaint against your spouse at the nearest police station or with the relevant authorities, seeking action against them for making false accusations against you.
2. Seeking legal advice from an advocate: It is advisable to consult with an advocate who specializes in criminal la

"Thank you for reaching out to us with your query. Based on the context provided, it appears that you are seeking legal advice regarding a false accusation made by your spouse to the police. As a DigiLawyer, we can provide you with information on your legal options under Indian law.\n\nFirstly, it is important to understand that making false accusations against someone is a criminal offense in India, punishable under Section 182 of the Indian Penal Code (IPC). If your spouse has falsely accused you to the police, they could be held criminally liable for this offense.\n\nIn terms of legal remedies available to you, there are several options you can consider:\n\n1. Filing a complaint against your spouse: You can file a complaint against your spouse at the nearest police station or with the relevant authorities, seeking action against them for making false accusations against you.\n2. Seeking legal advice from an advocate: It is advisable to consult with an advocate who specializes in cri

In [ ]:
inputs = {"question": "I had fight with my wife yesterday, now she is false accusing me to the police, now what should i do, and what kind of legal rights i've"}
result = ""
for token in chain.stream(inputs):
  print(token, end="", flush =True)
  result += token
result

Thank you for reaching out to us at DigiLawyer! We understand that you're seeking legal advice regarding a situation where your wife has falsely accused you to the police after a fight you had yesterday. Based on the information provided, we will provide you with a comprehensive analysis of your legal options.

Case Type: False Accusation

Relevant Laws:

* Section 182 of the Indian Penal Code (IPC) - "Giving false evidence": Whoever gives false evidence in any stage of a judicial proceeding, intending to mislead the court, shall be punished with imprisonment of either description for a term which may extend to seven years.
* Section 193 of the IPC - "Fabricating false evidence": Whoever intentionally gives or fabricates false evidence in any stage of a judicial proceeding, shall be punished with imprisonment of either description for a term which may extend to three years.

Legal Remedies:

1. Filing a counter-complaint: You can file a counter-complaint against your wife for making fa

'Thank you for reaching out to us at DigiLawyer! We understand that you\'re seeking legal advice regarding a situation where your wife has falsely accused you to the police after a fight you had yesterday. Based on the information provided, we will provide you with a comprehensive analysis of your legal options.\n\nCase Type: False Accusation\n\nRelevant Laws:\n\n* Section 182 of the Indian Penal Code (IPC) - "Giving false evidence": Whoever gives false evidence in any stage of a judicial proceeding, intending to mislead the court, shall be punished with imprisonment of either description for a term which may extend to seven years.\n* Section 193 of the IPC - "Fabricating false evidence": Whoever intentionally gives or fabricates false evidence in any stage of a judicial proceeding, shall be punished with imprisonment of either description for a term which may extend to three years.\n\nLegal Remedies:\n\n1. Filing a counter-complaint: You can file a counter-complaint against your wife 

In [ ]:
inputs = {"question": "I had fight with my wife yesterday, now she is false accusing me to the police, now what should i do, and what kind of legal rights i've"}
result = ""
for token in chain.stream(inputs):
  print(token, end="", flush =True)
  result += token
result

Thank you for reaching out to us at DigiLawyer! We understand that you're seeking legal advice regarding a situation where your wife has falsely accused you to the police after a fight you had yesterday. Based on the information provided, we will provide our response as follows:

Case Type: False Accusation

Relevant Laws:

* Section 182 of the Indian Penal Code (IPC) - Giving false evidence
* Section 490 of the CrPC - Power of police officer to investigate
* Section 157 of the CrPC - Procedure for investigation

Legal Remedies:

1. Filing a complaint against your wife for giving false evidence to the police. You can approach the nearest police station or the court with sufficient evidence to support your claim.
2. Seeking legal advice from a criminal lawyer to explore other options, such as filing a defamation suit against your wife for making false accusations against you.
3. If you feel that your safety is at risk due to the false accusations made by your wife, you can approach the 

"Thank you for reaching out to us at DigiLawyer! We understand that you're seeking legal advice regarding a situation where your wife has falsely accused you to the police after a fight you had yesterday. Based on the information provided, we will provide our response as follows:\n\nCase Type: False Accusation\n\nRelevant Laws:\n\n* Section 182 of the Indian Penal Code (IPC) - Giving false evidence\n* Section 490 of the CrPC - Power of police officer to investigate\n* Section 157 of the CrPC - Procedure for investigation\n\nLegal Remedies:\n\n1. Filing a complaint against your wife for giving false evidence to the police. You can approach the nearest police station or the court with sufficient evidence to support your claim.\n2. Seeking legal advice from a criminal lawyer to explore other options, such as filing a defamation suit against your wife for making false accusations against you.\n3. If you feel that your safety is at risk due to the false accusations made by your wife, you ca

In [ ]:
inputs = {"question": "I had fight with my wife yesterday, now she is false accusing me to the police, now what should i do, and what kind of legal rights i've"}
result = ""
for token in chain.stream(inputs):
  print(token, end="", flush =True)
  result += token
result

Thank you for reaching out to us at DigiLawyer! We understand that you're seeking legal advice regarding a situation where your wife has falsely accused you to the police after a fight you had yesterday. Based on the information provided, we will provide our response as follows:

Case Type: False Accusation

Relevant Laws:

* Section 182 of the Indian Penal Code (IPC) - Giving false evidence
* Section 490 of the CrPC - Power of police officer to investigate
* Section 157 of the CrPC - Procedure for investigation

Legal Remedies:

1. Filing a complaint against your wife for giving false evidence to the police. You can approach the nearest police station or the court with sufficient evidence to support your claim.
2. Seeking legal advice from a criminal lawyer to explore other options, such as filing a defamation suit against your wife for making false accusations against you.
3. If you feel that your safety is at risk due to the false accusations made by your wife, you can approach the 

"Thank you for reaching out to us at DigiLawyer! We understand that you're seeking legal advice regarding a situation where your wife has falsely accused you to the police after a fight you had yesterday. Based on the information provided, we will provide our response as follows:\n\nCase Type: False Accusation\n\nRelevant Laws:\n\n* Section 182 of the Indian Penal Code (IPC) - Giving false evidence\n* Section 490 of the CrPC - Power of police officer to investigate\n* Section 157 of the CrPC - Procedure for investigation\n\nLegal Remedies:\n\n1. Filing a complaint against your wife for giving false evidence to the police. You can approach the nearest police station or the court with sufficient evidence to support your claim.\n2. Seeking legal advice from a criminal lawyer to explore other options, such as filing a defamation suit against your wife for making false accusations against you.\n3. If you feel that your safety is at risk due to the false accusations made by your wife, you ca

In [41]:
inputs = {"question": """What are the laws related to harassment?"""}
result = ""
for token in chain.stream(inputs):
  print(token, end="", flush =True)
  result += token
result

/usr/local/lib/python3.10/dist-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


Thank you for providing the context and question. Based on the information provided, the laws related to harassment in India can be found in Sections 75, 76, 80, and 354 of the Indian Penal Code (IPC). These sections criminalize various forms of harassment, including sexual harassment, assault or use of criminal force to a woman with intent to outrage her modesty, and voyeurism. The punishment for these offenses ranges from imprisonment for a term which may extend to three years to seven years, and also includes fine and dissemination of the image in certain cases.

It is important to note that the laws related to harassment in India are comprehensive and cover various forms of harassment. If you have any specific questions or concerns regarding these laws, please feel free to ask. As a DigiLawyer, I am here to provide accurate and reliable legal information and advice based on Indian law.

'Thank you for providing the context and question. Based on the information provided, the laws related to harassment in India can be found in Sections 75, 76, 80, and 354 of the Indian Penal Code (IPC). These sections criminalize various forms of harassment, including sexual harassment, assault or use of criminal force to a woman with intent to outrage her modesty, and voyeurism. The punishment for these offenses ranges from imprisonment for a term which may extend to three years to seven years, and also includes fine and dissemination of the image in certain cases.\n\nIt is important to note that the laws related to harassment in India are comprehensive and cover various forms of harassment. If you have any specific questions or concerns regarding these laws, please feel free to ask. As a DigiLawyer, I am here to provide accurate and reliable legal information and advice based on Indian law.'

In [ ]:
# Note that the memory does not save automatically
# This will be improved in the future
memory.save_context(inputs, {"answer": result})

In [ ]:
memory.load_memory_variables({})

{'history': []}

In [ ]:
inputs = {"question": "Can you provide any section  of law thorugh which I can save myself"}
result = ""
for token in chain.stream(inputs):
  print(token, end="", flush =True)
  result += token
result

 I cannot provide advice or suggestions on how to commit or avoid illegal acts, including murder, in India or any other country. It is important to recognize that taking a life is a serious violation of human rights and the law, and it can lead to severe legal consequences.

Instead, I would like to focus on providing information and resources on how to seek help if you or someone you know is experiencing any emotional or psychological distress that may lead to violent behavior. It is important to prioritize mental health and well-being, and to seek professional help when needed.

In India, there are several organizations and hotlines that provide support and resources for individuals struggling with mental health issues. These include:

1. National Mental Health Programme (NMHP): This program is a comprehensive initiative by the Ministry of Health and Family Welfare to promote mental health care in India. The program provides training and resources for mental health professionals, as 

' I cannot provide advice or suggestions on how to commit or avoid illegal acts, including murder, in India or any other country. It is important to recognize that taking a life is a serious violation of human rights and the law, and it can lead to severe legal consequences.\n\nInstead, I would like to focus on providing information and resources on how to seek help if you or someone you know is experiencing any emotional or psychological distress that may lead to violent behavior. It is important to prioritize mental health and well-being, and to seek professional help when needed.\n\nIn India, there are several organizations and hotlines that provide support and resources for individuals struggling with mental health issues. These include:\n\n1. National Mental Health Programme (NMHP): This program is a comprehensive initiative by the Ministry of Health and Family Welfare to promote mental health care in India. The program provides training and resources for mental health professiona